In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Construir circuitos

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Versiones de paquetes</b></summary>

El código en esta página se desarrolló utilizando los siguientes requisitos.
Recomendamos usar estas versiones o versiones más recientes.

```
qiskit[all]~=2.3.0
```
</details>
Esta página examina más de cerca la clase [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) en el SDK de Qiskit, incluidos algunos métodos más avanzados que puede usar para crear circuitos cuánticos.
## ¿Qué es un circuito cuántico?
Un circuito cuántico simple es una colección de qubits y una lista de instrucciones que actúan sobre esos qubits. Para demostrarlo, la siguiente celda crea un nuevo circuito con dos nuevos qubits, luego muestra el atributo [`qubits`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qubits) del circuito, que es una lista de [`Qubits`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Qubit) en orden desde el bit menos significativo $q_0$ hasta el bit más significativo $q_n$.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.qubits

[<Qubit register=(2, "q"), index=0>, <Qubit register=(2, "q"), index=1>]

Multiple `QuantumRegister` and `ClassicalRegister` objects can be combined to create a circuit. Every [`QuantumRegister`](/docs/api/qiskit/circuit#qiskit.circuit.QuantumRegister) and [`ClassicalRegister`](/docs/api/qiskit/circuit#qiskit.circuit.ClassicalRegister) can also be named.

In [2]:
from qiskit.circuit import QuantumRegister, ClassicalRegister

qr1 = QuantumRegister(2, "qreg1")  # Create a QuantumRegister with 2 qubits
qr2 = QuantumRegister(1, "qreg2")  # Create a QuantumRegister with 1 qubit
cr1 = ClassicalRegister(3, "creg1")  # Create a ClassicalRegister with 3 cbits

combined_circ = QuantumCircuit(
    qr1, qr2, cr1
)  # Create a quantum circuit with 2 QuantumRegisters and 1 ClassicalRegister
combined_circ.qubits

[<Qubit register=(2, "qreg1"), index=0>,
 <Qubit register=(2, "qreg1"), index=1>,
 <Qubit register=(1, "qreg2"), index=0>]

Se pueden combinar múltiples objetos `QuantumRegister` y `ClassicalRegister` para crear un circuito. Cada [`QuantumRegister`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.QuantumRegister) y [`ClassicalRegister`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.ClassicalRegister) también puede ser nombrado.

In [3]:
desired_qubit = qr2[0]  # Qubit 0 of register 'qreg2'

print("Index:", combined_circ.find_bit(desired_qubit).index)
print("Register:", combined_circ.find_bit(desired_qubit).registers)

Index: 2
Register: [(QuantumRegister(1, 'qreg2'), 0)]


Adding an instruction to the circuit appends the instruction to the circuit's [`data`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#data) attribute. The following cell output shows `data` is a list of [`CircuitInstruction`](/docs/api/qiskit/qiskit.circuit.CircuitInstruction) objects, each of which has an `operation` attribute, and a `qubits` attribute.

In [4]:
qc.x(0)  # Add X-gate to qubit 0
qc.data

[CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(2, "q"), index=0>,), clbits=())]

Puede encontrar el índice y el registro de un qubit usando el método [`find_bit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.find_bit) del circuito y sus atributos.

In [5]:
qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/43a57258-3e33-4071-8a48-2bf127c8a5be-0.svg" alt="Output of the previous code cell" />

Circuit instruction objects can contain "definition" circuits that describe the instruction in terms of more fundamental instructions. For example, the [X-gate](/docs/api/qiskit/qiskit.circuit.library.XGate) is defined as a specific case of the [U3-gate](/docs/api/qiskit/qiskit.circuit.library.U3Gate), a more general single-qubit gate.

In [6]:
# Draw definition circuit of 0th instruction in `qc`
qc.data[0].operation.definition.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/653e2427-e301-4d2f-84de-1959185ace8e-0.svg" alt="Output of the previous code cell" />

Agregar una instrucción al circuito anexa la instrucción al atributo [`data`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#data) del circuito. La siguiente salida de celda muestra que `data` es una lista de objetos [`CircuitInstruction`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.CircuitInstruction), cada uno de los cuales tiene un atributo `operation` y un atributo `qubits`.

In [7]:
from qiskit.circuit.library import HGate

qc = QuantumCircuit(1)
qc.append(
    HGate(),  # New HGate instruction
    [0],  # Apply to qubit 0
)
qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/66813cae-9841-47ea-96b7-8fd7b82e9759-0.svg" alt="Output of the previous code cell" />

To combine two circuits, use the [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method. This accepts another [`QuantumCircuit`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit) and an optional list of qubit mappings.

<Admonition type="note">
    The [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method returns a new circuit and does **not** mutate either circuit it acts on. To mutate the circuit on which you're calling the [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method, use the argument `inplace=True`.
</Admonition>

In [8]:
qc_a = QuantumCircuit(4)
qc_a.x(0)

qc_b = QuantumCircuit(2, name="qc_b")
qc_b.y(0)
qc_b.z(1)

# compose qubits (0, 1) of qc_a to qubits (1, 3) of qc_b respectively
combined = qc_a.compose(qc_b, qubits=[1, 3])
combined.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/29152dfa-2275-4bc4-aadb-82185b9e0e86-0.svg" alt="Output of the previous code cell" />

La forma más fácil de ver esta información es a través del método [`draw`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#draw), que devuelve una visualización de un circuito. Consulte [Visualizar circuitos](./visualize-circuits) para diferentes formas de mostrar circuitos cuánticos.

In [9]:
inst = qc_b.to_instruction()
qc_a.append(inst, [1, 3])
qc_a.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/81b682dd-45cb-4492-809e-d9e8ebbf5600-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/43a57258-3e33-4071-8a48-2bf127c8a5be-0.svg)

Los objetos de instrucción de circuito pueden contener circuitos de "definición" que describen la instrucción en términos de instrucciones más fundamentales. Por ejemplo, la [compuerta X](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.XGate) se define como un caso específico de la [compuerta U3](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.U3Gate), una compuerta de un solo qubit más general.

In [10]:
gate = qc_b.to_gate().control()
qc_a.append(gate, [0, 1, 3])
qc_a.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/ed362e64-d6a4-4dfd-a5cf-5e6bdc7a81b5-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/653e2427-e301-4d2f-84de-1959185ace8e-0.svg)

Las instrucciones y los circuitos son similares en que ambos describen operaciones sobre bits y qubits, pero tienen propósitos diferentes:

- Las instrucciones se tratan como fijas, y sus métodos generalmente devolverán nuevas instrucciones (sin mutar el objeto original).
- Los circuitos están diseñados para construirse a lo largo de muchas líneas de código, y los métodos [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) a menudo mutan el objeto existente.
### ¿Qué es la profundidad del circuito?
La [profundidad (depth)](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.depth) de un circuito cuántico es una medida del número de "capas" de compuertas cuánticas, ejecutadas en paralelo, que se necesitan para completar el cálculo definido por el circuito. Debido a que las compuertas cuánticas toman tiempo para implementarse, la profundidad de un circuito corresponde aproximadamente a la cantidad de tiempo que tarda la computadora cuántica en ejecutar el circuito. Por lo tanto, la profundidad de un circuito es una cantidad importante que se usa para medir si un circuito cuántico puede ejecutarse en un dispositivo.

El resto de esta página ilustra cómo manipular circuitos cuánticos.
## Construir circuitos
Métodos como [`QuantumCircuit.h`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#h) y [`QuantumCircuit.cx`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#cx) agregan instrucciones específicas a los circuitos. Para agregar instrucciones a un circuito de manera más general, use el método [`append`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#append). Esto toma una instrucción y una lista de qubits a los que aplicar la instrucción. Consulte la [documentación de la API de Circuit Library](https://docs.quantum.ibm.com/api/qiskit/circuit_library) para obtener una lista de instrucciones compatibles.

In [11]:
qc_a.decompose().draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/3c0633db-929b-4428-a888-7a3d493bd6dd-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/66813cae-9841-47ea-96b7-8fd7b82e9759-0.svg)

Para combinar dos circuitos, use el método [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose). Este acepta otro [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) y una lista opcional de mapeos de qubits.

> **Note:** El método [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose) devuelve un nuevo circuito y **no** muta ninguno de los circuitos sobre los que actúa. Para mutar el circuito sobre el que está llamando al método [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose), use el argumento `inplace=True`.

In [12]:
qc1 = QuantumCircuit(2, 2)
qc1.measure(0, 1)
qc1.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/0cdb2273-0.svg" alt="Output of the previous code cell" />

In [13]:
qc2 = QuantumCircuit(2)
qc2.measure_all()
qc2.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/6f33698c-0.svg" alt="Output of the previous code cell" />

In [14]:
qc3 = QuantumCircuit(2)
qc3.x(1)
qc3.measure_active()
qc3.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/ca3f225f-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/ed362e64-d6a4-4dfd-a5cf-5e6bdc7a81b5-0.svg)

Para ver qué está sucediendo, puede usar el método [`decompose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#decompose) para expandir cada instrucción en su definición.

> **Note:** El método [`decompose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#decompose) devuelve un nuevo circuito y **no** muta el circuito sobre el que actúa.

In [15]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit import Parameter

angle = Parameter("angle")  # undefined number

# Create and optimize circuit once
qc = QuantumCircuit(1)
qc.rx(angle, 0)
qc = generate_preset_pass_manager(
    optimization_level=3, basis_gates=["u", "cx"]
).run(qc)

qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/a580552c-d585-4047-99f0-32aafd06e4f3-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/3c0633db-929b-4428-a888-7a3d493bd6dd-0.svg)

<span id="measure-qubits"></span>
## Medir qubits
Las mediciones se utilizan para muestrear los estados de qubits individuales y transferir los resultados a un registro clásico. Tenga en cuenta que si está enviando circuitos a una primitiva [Sampler](./primitives#sampler), se requieren mediciones. Sin embargo, los circuitos enviados a una primitiva [Estimator](./primitives#estimator) no deben contener mediciones.

Los qubits se pueden medir usando tres métodos: [`measure`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.measure), [`measure_all`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) y [`measure_active`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_active). Para aprender cómo visualizar resultados medidos, consulte la página [Visualizar resultados](./visualize-results).

1. `QuantumCircuit.measure`: mide cada qubit en el primer argumento en el bit clásico dado como segundo argumento. Este método permite un control total sobre dónde se almacena el resultado de la medición.

2. `QuantumCircuit.measure_all`: no toma argumentos y se puede usar para circuitos cuánticos sin bits clásicos predefinidos. Crea cables clásicos y almacena los resultados de la medición en orden. Por ejemplo, la medición del qubit $q_i$ se almacena en cbit $meas_i$). También agrega una barrera antes de la medición.

3. `QuantumCircuit.measure_active`: similar a `measure_all`, pero mide solo los qubits que tienen operaciones.

In [16]:
circuits = []
for value in range(100):
    circuits.append(qc.assign_parameters({angle: value}))

circuits[0].draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/85af6231-921a-4130-99d3-f6998f761df8-0.svg" alt="Output of the previous code cell" />

You can find a list of a circuit's undefined parameters in its `parameters` attribute.

In [17]:
qc.parameters

ParameterView([Parameter(angle)])

### Change a parameter's name

By default, parameter names for a parameterized circuit are prefixed by `x`- for example, `x[0]`. You can change the names after they are defined, as shown in the following example.

In [18]:
from qiskit.circuit.library import z_feature_map
from qiskit.circuit import ParameterVector

# Define a parameterized circuit with default names
# For example, x[0]
circuit = z_feature_map(2)

# Set new parameter names
# They will now be prefixed by `hi` instead
# For example, hi[0]
training_params = ParameterVector("hi", 2)

# Assign parameter names to the quantum circuit
circuit = circuit.assign_parameters(parameters=training_params)

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/ca3f225f-0.svg)

## Circuitos parametrizados

Muchos algoritmos cuánticos a corto plazo implican ejecutar muchas variaciones de un circuito cuántico. Dado que construir y optimizar circuitos grandes puede ser computacionalmente costoso, Qiskit admite circuitos **parametrizados**. Estos circuitos tienen parámetros indefinidos, y sus valores no necesitan definirse hasta justo antes de ejecutar el circuito. Esto le permite mover la construcción y optimización del circuito fuera del bucle del programa principal. La siguiente celda crea y muestra un circuito parametrizado.